# Semantic Scholar API

In [ ]:
import httpx

In [ ]:
doi = "10.1101/444398"

In [ ]:
r1 = httpx.get(
    f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}?fields=title,url,abstract,authors"
)

In [ ]:
def print_paper(paper: dict):
    print(paper["title"])
    print(paper["url"])
    print(paper["abstract"])
    print(paper["authors"])

In [ ]:
j1 = r1.json()
print_paper(j1)

In [ ]:
paper_id = j1["paperId"]

In [ ]:
num_papers = 100

In [ ]:
r2 = httpx.get(
    f"https://api.semanticscholar.org/recommendations/v1/papers/forpaper/{paper_id}?limit={num_papers}&fields=title,url,abstract,authors"
)  # ")

In [ ]:
j2 = r2.json()

In [ ]:
ids = [p["paperId"] for p in j2["recommendedPapers"]]
payload = {"ids": ids}

In [ ]:
r3 = httpx.post(
    "https://api.semanticscholar.org/graph/v1/paper/batch?fields=title,isOpenAccess,openAccessPdf,authors,embedding,tldr",
    json=payload, timeout=50.0
)

In [ ]:
j3 = r3.json()

In [ ]:
payload

In [ ]:
id_to_vector = {}
id_to_title = {}
for p in j3:
    id_ = p["paperId"]
    assert p["embedding"]["model"] == "specter@v0.1.1"
    id_to_vector[id_] = p["embedding"]["vector"]
    id_to_title[id_] = p["title"]

### UMAP embedding

In [ ]:
import bokeh
import pandas as pd
import umap

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10

output_notebook()

In [ ]:
reducer = umap.UMAP(random_state=42)

In [ ]:
df = pd.DataFrame(id_to_vector).transpose()
df

In [ ]:
reducer.fit(df)

In [ ]:
df_t = pd.DataFrame(reducer.transform(df))

In [ ]:
df_t["id_"] = df.index.copy()
df_t["title"] = [id_to_title[i] for i in df_t["id_"]]
df_t

In [ ]:
datasource = ColumnDataSource({str(c): v.values for c, v in df_t.items()})

In [ ]:
tooltips = [
    ("(x,y)", "($x, $y)"),
    ("id", "@id_"),
    ("title", "@title"),
]

In [ ]:
plot_figure = figure(
    title='UMAP projection of papers',
    width=800,
    height=800,
    tooltips=tooltips,
)

In [ ]:
plot_figure.circle(
    '0',
    '1',
    source=datasource,
    line_alpha=0.6,
    fill_alpha=0.6,
    size=4
)
show(plot_figure)

### BioGPT

In [ ]:
from transformers import BioGptForCausalLM, BioGptTokenizer, pipeline, set_seed

In [ ]:
model = BioGptForCausalLM.from_pretrained("microsoft/biogpt-large")
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt-large")

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
set_seed(42)
generator("COVID-19 is", max_length=20, num_return_sequences=5, do_sample=True)

In [ ]:
input_text = f"Task: please summarize the following article. Article: {j1['abstract']}"
input_text += " Summary: "
input_text

In [ ]:
generator(input_text, max_new_tokens=200, num_return_sequences=5, do_sample=True)